In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/statoil-iceberg-classifier-challenge/train.json.7z
/kaggle/input/statoil-iceberg-classifier-challenge/test.json.7z
/kaggle/input/statoil-iceberg-classifier-challenge/sample_submission.csv.7z


In [2]:
!pip install py7zr
import py7zr
import os

if not os.path.exists('/kaggle/train/') :
    os.makedirs('/kaggle/train/')

if not os.path.exists('/kaggle/test/') :
    os.makedirs('/kaggle/test/')

with py7zr.SevenZipFile("/kaggle/input/statoil-iceberg-classifier-challenge/train.json.7z", 'r') as archive:
    archive.extractall(path="/kaggle/train")

with py7zr.SevenZipFile("/kaggle/input/statoil-iceberg-classifier-challenge/test.json.7z", 'r') as archive:
    archive.extractall(path="/kaggle/test")

for dirname, _, filenames in os.walk('/kaggle'): 
    for filename in filenames: 
        print(os.path.join(dirname, filename))

     |████████████████████████████████| 65 kB 583 kB/s 
     |████████████████████████████████| 2.2 MB 1.1 MB/s 
     |████████████████████████████████| 121 kB 6.1 MB/s 
     |████████████████████████████████| 1.9 MB 5.9 MB/s 
     |████████████████████████████████| 357 kB 7.6 MB/s 
/kaggle/src/script.ipynb
/kaggle/lib/kaggle/gcp.py
/kaggle/test/data/processed/test.json
/kaggle/input/statoil-iceberg-classifier-challenge/train.json.7z
/kaggle/input/statoil-iceberg-classifier-challenge/test.json.7z
/kaggle/input/statoil-iceberg-classifier-challenge/sample_submission.csv.7z
/kaggle/train/data/processed/train.json
/kaggle/working/__notebook__.ipynb


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize']= 10,10
%matplotlib inline

In [4]:
train = pd.read_json('/kaggle/train/data/processed/train.json')
target_train = train['is_iceberg']
test = pd.read_json('/kaggle/test/data/processed/test.json')

In [5]:
target_train

0       0
1       0
2       1
3       0
4       0
       ..
1599    0
1600    0
1601    0
1602    0
1603    0
Name: is_iceberg, Length: 1604, dtype: int64

Keras provide the implementation of pretrained VGG, so don't have to build the net by ourselves.
Here we are removing the last layer of VGG and putting our sigmoid layer for binary predictions.

In [6]:
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')

train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

In [7]:
X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train["band_1"]])
X_band_2 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train["band_2"]])
X_band_3 = (X_band_1+X_band_2)/2

X_train = np.concatenate([X_band_1[:,:,:,np.newaxis], X_band_2[:,:,:,np.newaxis], X_band_3[:,:,:,np.newaxis]], axis=-1)

In [8]:
X_band_test_1 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in test["band_1"]])
X_band_test_2 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in test["band_2"]])
X_band_test_3 = (X_band_test_1+X_band_test_2)/2

X_test = np.concatenate([X_band_test_1[:,:,:,np.newaxis], X_band_test_2[:,:,:,np.newaxis], X_band_test_3[:,:,:,np.newaxis]], axis=-1)

In [9]:
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input


In [10]:
# Data Augmentation for multi-input
from keras.preprocessing.image import ImageDataGenerator
batch_size = 64
# Define the image transformations
gen = ImageDataGenerator(

    horizontal_flip = True,
    vertical_flip = True,
    width_shift_range = 0.,
    height_shift_range = 0.,
    channel_shift_range =0,
    zoom_range = 0.2,
    rotation_range = 10
    
)

In [11]:
# Function that merges two genetators
def gen_flow_for_two_inputs(X1,X2,y):
    genX1 = gen.flow(X1, y, batch_size=batch_size, seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size, seed=55)
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        # Assert arrays are equal, peace of mind, but slow down training
        # np.testing.assert_array_equal(X1i[0],X2i[0])
        yield [X1i[0], X2i[1]], X1i[1]

        
# Function that create generator
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=10, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es,msave]

def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    
    x = base_model.get_layer('block5_pool').output
    
    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model([base_model.input, input_2], predictions)
    
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model



# using K-fold Cross Validation with Data Augmentation
def myAngleCV(X_train, X_angle, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log = 0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n=================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout = target_train[test_idx]
        
        X_angle_cv  = X_angle[train_idx]
        X_angle_hold = X_angle[test_idx]
        
        # define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        
        galaxyModel = getVggAngleModel()
        galaxyModel.fit_generator(
        
            gen_flow,
            steps_per_epoch = 24,
            epochs = 100,
            shuffle = True,
            verbose = 1,
            validation_data = ([X_holdout, X_angle_hold], Y_holdout), 
            callbacks=callbacks
            
        )
        
        # Getting the bast model
        galaxyModel.load_weights(filepath=file_path)
        # Getting training score
        score = galaxyModel.evaluate([X_train_cv, X_angle_cv], y_train_cv, verbose=0)
        print('Train Loss: ',score[0])
        print('Train Accuracy: ',score[1])
        
        # Getting test score
        score = galaxyModel.evaluate([X_holdout, X_angle_hold], Y_holdout, verbose=0)
        print('Test Loss: ',score[0])
        print('Test accuracy: ',score[1])
        
        # Getting validation score
        pred_valid = galaxyModel.predict([X_holdout, X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])
        
        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])
        
    y_test_pred_log = y_test_pred_log/K
    y_train_pred_log = y_train_pred_log/K
    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [12]:
preds = myAngleCV(X_train, X_angle, X_test)


=================FOLD= 0
58892288/58889256 [==============================] - 1s 0us/step


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
24/24 [==============================] - 153s 6s/step - loss: 0.7970 - accuracy: 0.5424 - val_loss: 0.5177 - val_accuracy: 0.7196
Epoch 2/100
24/24 [==============================] - 150s 6s/step - loss: 0.5092 - accuracy: 0.7146 - val_loss: 0.3365 - val_accuracy: 0.8299
Epoch 3/100
24/24 [==============================] - 148s 6s/step - loss: 0.3850 - accuracy: 0.8178 - val_loss: 0.2766 - val_accuracy: 0.8822
Epoch 4/100
24/24 [==============================] - 149s 6s/step - loss: 0.3043 - accuracy: 0.8523 - val_loss: 0.2822 - val_accuracy: 0.8785
Epoch 5/100
24/24 [==============================] - 147s 6s/step - loss: 0.3075 - accuracy: 0.8549 - val_loss: 0.2714 - val_accuracy: 0.8822
Epoch 6/100
24/24 [==============================] - 150s 6s/step - loss: 0.2951 - accuracy: 0.8535 - val_loss: 0.2499 - val_accuracy: 0.8841
Epoch 7/100
24/24 [==============================] - 148s 6s/step - loss: 0.2739 - accuracy: 0.8621 - val_loss: 0.2585 - val_accuracy: 0.8785
Epoch 

In [13]:
submission = pd.DataFrame()
submission['id'] = test['id']
submission['is_iceberg'] = preds
submission.to_csv('sub.csv', index=False)